# SOEN Hardware Primitives Exploration

This notebook visualizes how each hardware parameter affects SingleDendrite neuron behavior.

## The Core Equation

```
ds/dt = γ⁺ · g(φ, I_squid) - γ⁻ · s
```

Where:
- `s`: Stored current in the superconducting loop (neuron state)
- `φ`: Magnetic flux = J·s_in + φ_offset
- `I_squid`: SQUID current = I_bias - s
- `g(φ, I)`: Source function (SQUID response curve)
- `γ⁺`: Integration rate (photon-to-current gain)
- `γ⁻`: Decay rate (L/R time constant)

## Parameter Classification

| Parameter | Fixed At | Trainable? |
|-----------|----------|------------|
| g(φ) shape | Physics | No |
| φ_offset | Fabrication | No |
| I_bias | Fabrication | No |
| γ⁺, γ⁻ | Fabrication | No |
| J (weights) | Runtime | **Yes** |

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Import SOEN components
from soen_toolkit.core.source_functions import SOURCE_FUNCTIONS
from soen_toolkit.core.source_functions.heaviside import HeavisideFitStateDep

torch.manual_seed(42)
np.random.seed(42)

print("Available source functions:", list(SOURCE_FUNCTIONS.keys()))

---
## 1. The Source Function g(φ) - The Heart of SOEN

The source function maps magnetic flux to conductance. This is determined by SQUID physics.

In [ ]:
# Create source function
g_func = HeavisideFitStateDep()

# Plot g(φ) for different squid currents
phi_range = torch.linspace(-0.5, 1.5, 500)
squid_currents = [1.2, 1.5, 1.7, 1.9, 2.0]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: g(φ) curves for different I_squid
ax1 = axes[0]
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(squid_currents)))

for i_squid, color in zip(squid_currents, colors):
    i_tensor = torch.full_like(phi_range, i_squid)
    g_vals = g_func.g(phi_range, squid_current=i_tensor).numpy()
    ax1.plot(phi_range.numpy(), g_vals, color=color, lw=2, label=f'I_squid={i_squid}')

ax1.axvline(x=0.23, color='red', linestyle='--', alpha=0.7, label='φ=0.23 (threshold)')
ax1.axvline(x=0.5, color='gray', linestyle=':', alpha=0.5)
ax1.axvline(x=0, color='gray', linestyle=':', alpha=0.5)
ax1.axvline(x=1, color='gray', linestyle=':', alpha=0.5)
ax1.set_xlabel('Flux φ (units of Φ₀)')
ax1.set_ylabel('g(φ) - Conductance')
ax1.set_title('Source Function g(φ) for Different SQUID Currents')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-0.5, 1.5)

# Right: Key properties
ax2 = axes[1]
phi_one_period = torch.linspace(0, 1, 200)
i_squid_default = torch.full_like(phi_one_period, 1.7)
g_default = g_func.g(phi_one_period, squid_current=i_squid_default).numpy()

ax2.plot(phi_one_period.numpy(), g_default, 'b-', lw=2, label='g(φ)')
ax2.fill_between(phi_one_period.numpy(), 0, g_default, alpha=0.3)

# Mark key regions
ax2.axvspan(0, 0.15, alpha=0.2, color='green', label='Low φ: neuron OFF')
ax2.axvspan(0.15, 0.35, alpha=0.2, color='yellow', label='Transition zone')
ax2.axvspan(0.35, 0.5, alpha=0.2, color='red', label='High φ: neuron ON')

ax2.axvline(x=0.23, color='red', linestyle='--', lw=2, label='φ=0.23 threshold')
ax2.set_xlabel('Flux φ (units of Φ₀)')
ax2.set_ylabel('g(φ)')
ax2.set_title('One Period of g(φ) with Operating Regions')
ax2.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Properties of g(φ):")
print("  • Periodic with period Φ₀ (normalized to 1)")
print("  • Symmetric around φ = 0.5")
print("  • Peak near φ = 0.5, minimum near φ = 0 and φ = 1")
print("  • Shape determined by SQUID junction physics (FIXED)")

---
## 2. Effect of φ_offset (Operating Point)

`φ_offset` shifts where on the g(φ) curve the neuron operates. This is set at fabrication.

In [ ]:
# Different phi_offset values
phi_offsets = [0.02, 0.10, 0.15, 0.23, 0.30, 0.40]

# Input flux range (from upstream neurons via J)
phi_input = torch.linspace(-0.1, 0.3, 200)
i_squid = torch.full_like(phi_input, 1.7)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: g(φ_total) for different offsets
ax1 = axes[0]
colors = plt.cm.plasma(np.linspace(0.1, 0.9, len(phi_offsets)))

for offset, color in zip(phi_offsets, colors):
    phi_total = phi_input + offset
    g_vals = g_func.g(phi_total, squid_current=i_squid).numpy()
    ax1.plot(phi_input.numpy(), g_vals, color=color, lw=2, label=f'φ_offset={offset}')

ax1.axvline(x=0, color='black', linestyle='--', alpha=0.5, label='Zero input')
ax1.set_xlabel('Input Flux φ_in = J·s (from upstream)')
ax1.set_ylabel('g(φ_in + φ_offset)')
ax1.set_title('Effect of φ_offset on Input-Output Curve')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# Right: Sensitivity analysis
ax2 = axes[1]

# Compute derivative (sensitivity) at zero input
sensitivities = []
baseline_outputs = []
delta = 0.001

for offset in phi_offsets:
    phi_0 = torch.tensor([offset])
    phi_delta = torch.tensor([offset + delta])
    i_sq = torch.tensor([1.7])
    
    g_0 = g_func.g(phi_0, squid_current=i_sq).item()
    g_delta = g_func.g(phi_delta, squid_current=i_sq).item()
    
    sensitivity = (g_delta - g_0) / delta
    sensitivities.append(sensitivity)
    baseline_outputs.append(g_0)

x = np.arange(len(phi_offsets))
width = 0.35

bars1 = ax2.bar(x - width/2, baseline_outputs, width, label='Baseline g(φ_offset)', color='steelblue')
ax2_twin = ax2.twinx()
bars2 = ax2_twin.bar(x + width/2, sensitivities, width, label='Sensitivity dg/dφ', color='coral')

ax2.set_xticks(x)
ax2.set_xticklabels([f'{o}' for o in phi_offsets])
ax2.set_xlabel('φ_offset')
ax2.set_ylabel('Baseline Output g(φ_offset)', color='steelblue')
ax2_twin.set_ylabel('Sensitivity dg/dφ', color='coral')
ax2.set_title('Baseline Output vs Sensitivity at Different φ_offset')
ax2.legend(loc='upper left')
ax2_twin.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\nφ_offset Design Considerations:")
print("  • Low φ_offset (0.02): Neuron mostly OFF, needs strong input to activate")
print("  • φ_offset ≈ 0.23: At threshold, maximum sensitivity to small inputs")
print("  • High φ_offset (0.40): Neuron mostly ON, saturated response")
print("\n  → For training, φ_offset=0.23 is ideal (gradient flows well)")

---
## 3. Effect of I_bias (SQUID Current)

`I_bias` sets the DC bias current. The SQUID current `I_squid = I_bias - s` affects the g(φ) curve height.

In [ ]:
# Different bias currents
bias_currents = [1.3, 1.5, 1.7, 1.9, 2.1]

# Fixed phi range
phi_range = torch.linspace(0, 0.5, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: g(φ) curves for different bias currents (at s=0)
ax1 = axes[0]
colors = plt.cm.coolwarm(np.linspace(0.1, 0.9, len(bias_currents)))

for i_bias, color in zip(bias_currents, colors):
    # At s=0, I_squid = I_bias
    i_squid = torch.full_like(phi_range, i_bias)
    g_vals = g_func.g(phi_range, squid_current=i_squid).numpy()
    ax1.plot(phi_range.numpy(), g_vals, color=color, lw=2, label=f'I_bias={i_bias}')

ax1.set_xlabel('Flux φ')
ax1.set_ylabel('g(φ, I_squid=I_bias)')
ax1.set_title('Effect of I_bias on g(φ) Curve (at s=0)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: How stored current s affects I_squid dynamically
ax2 = axes[1]

# Simulate how I_squid changes as s increases
s_values = np.linspace(0, 1.5, 100)
phi_fixed = 0.23  # At threshold

for i_bias, color in zip(bias_currents, colors):
    g_vals = []
    for s in s_values:
        i_squid = i_bias - s  # I_squid = I_bias - s
        if i_squid < 0:
            i_squid = 0.01  # Clamp
        phi_t = torch.tensor([phi_fixed])
        i_t = torch.tensor([i_squid])
        g_val = g_func.g(phi_t, squid_current=i_t).item()
        g_vals.append(g_val)
    ax2.plot(s_values, g_vals, color=color, lw=2, label=f'I_bias={i_bias}')

ax2.set_xlabel('Stored Current s')
ax2.set_ylabel('g(φ=0.23, I_squid=I_bias-s)')
ax2.set_title('State-Dependent Gain: g decreases as s increases')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nI_bias Design Considerations:")
print("  • Higher I_bias → Higher peak g(φ) → Faster integration")
print("  • I_squid = I_bias - s creates NEGATIVE FEEDBACK:")
print("    - As s increases, I_squid decreases")
print("    - Lower I_squid means lower g(φ)")
print("    - This provides natural gain control / saturation")
print("\n  → I_bias sets the 'headroom' before saturation")

---
## 4. Effect of γ⁺ and γ⁻ (Time Constants)

These determine how fast the neuron integrates input and decays.

In [ ]:
def simulate_ode(phi_input, phi_offset, i_bias, gamma_plus, gamma_minus, dt, n_steps):
    """
    Simulate ds/dt = γ⁺·g(φ) - γ⁻·s using Forward Euler.
    """
    s = 0.0  # Initial state
    s_history = [s]
    
    for t in range(n_steps):
        # Get input at time t
        phi_t = phi_input[t] if hasattr(phi_input, '__len__') else phi_input
        phi_total = phi_t + phi_offset
        
        # Compute I_squid
        i_squid = max(i_bias - s, 0.01)
        
        # Compute g(φ)
        phi_tensor = torch.tensor([phi_total])
        i_tensor = torch.tensor([i_squid])
        g_val = g_func.g(phi_tensor, squid_current=i_tensor).item()
        
        # ODE: ds/dt = γ⁺·g - γ⁻·s
        dsdt = gamma_plus * g_val - gamma_minus * s
        
        # Forward Euler step
        s = s + dt * dsdt
        s_history.append(s)
    
    return np.array(s_history)


# Simulation parameters
dt = 1.0
n_steps = 500
time = np.arange(n_steps + 1) * dt

# Create step input
phi_input = np.zeros(n_steps)
phi_input[50:] = 0.15  # Step input at t=50

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# (0,0): Effect of γ⁺
ax = axes[0, 0]
gamma_plus_values = [0.0001, 0.0005, 0.001, 0.002, 0.005]
colors = plt.cm.Greens(np.linspace(0.3, 0.9, len(gamma_plus_values)))

for gp, color in zip(gamma_plus_values, colors):
    s_hist = simulate_ode(phi_input, phi_offset=0.23, i_bias=1.98, 
                          gamma_plus=gp, gamma_minus=1e-6, dt=dt, n_steps=n_steps)
    ax.plot(time, s_hist, color=color, lw=2, label=f'γ⁺={gp}')

ax.axvline(x=50, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Time')
ax.set_ylabel('State s')
ax.set_title('Effect of γ⁺ (Integration Rate)')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# (0,1): Effect of γ⁻
ax = axes[0, 1]
gamma_minus_values = [1e-6, 1e-5, 1e-4, 5e-4, 1e-3]
colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(gamma_minus_values)))

for gm, color in zip(gamma_minus_values, colors):
    s_hist = simulate_ode(phi_input, phi_offset=0.23, i_bias=1.98,
                          gamma_plus=0.0005, gamma_minus=gm, dt=dt, n_steps=n_steps)
    ax.plot(time, s_hist, color=color, lw=2, label=f'γ⁻={gm:.0e}')

ax.axvline(x=50, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Time')
ax.set_ylabel('State s')
ax.set_title('Effect of γ⁻ (Decay Rate)')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# (1,0): Pulse response
ax = axes[1, 0]

# Create pulse input
phi_pulse = np.zeros(n_steps)
phi_pulse[50:100] = 0.15  # Pulse from t=50 to t=100

# Different γ⁺/γ⁻ ratios
ratios = [
    (0.001, 1e-6, 'Integrator (γ⁺>>γ⁻)'),
    (0.001, 1e-4, 'Balanced'),
    (0.001, 1e-3, 'Leaky (γ⁺≈γ⁻)'),
]
colors = ['blue', 'green', 'red']

for (gp, gm, label), color in zip(ratios, colors):
    s_hist = simulate_ode(phi_pulse, phi_offset=0.23, i_bias=1.98,
                          gamma_plus=gp, gamma_minus=gm, dt=dt, n_steps=n_steps)
    ax.plot(time, s_hist, color=color, lw=2, label=label)

# Show input pulse
ax.fill_between([50, 100], 0, ax.get_ylim()[1] * 0.1, alpha=0.3, color='gray', label='Input pulse')
ax.set_xlabel('Time')
ax.set_ylabel('State s')
ax.set_title('Pulse Response: γ⁺/γ⁻ Ratio Determines Memory')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# (1,1): Time constant visualization
ax = axes[1, 1]

# Decay from initial state (no input)
phi_zero = np.zeros(n_steps)

for gm, color in zip([1e-5, 1e-4, 5e-4, 1e-3], plt.cm.Oranges(np.linspace(0.3, 0.9, 4))):
    # Start with s=1.0 and let it decay
    s = 1.0
    s_hist = [s]
    for _ in range(n_steps):
        # No input, just decay: ds/dt = -γ⁻·s
        s = s * (1 - dt * gm)
        s_hist.append(s)
    
    tau = 1/gm  # Time constant
    ax.plot(time, s_hist, color=color, lw=2, label=f'γ⁻={gm:.0e}, τ={tau:.0f}')

ax.axhline(y=1/np.e, color='black', linestyle='--', alpha=0.5, label='1/e ≈ 0.37')
ax.set_xlabel('Time')
ax.set_ylabel('State s')
ax.set_title('Decay Time Constant τ = 1/γ⁻')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nγ⁺ and γ⁻ Design Considerations:")
print("  • γ⁺ (integration rate): How fast neuron responds to input")
print("  • γ⁻ (decay rate): How fast neuron forgets (time constant τ = 1/γ⁻)")
print("  • γ⁺/γ⁻ ratio determines behavior:")
print("    - High ratio → Integrator (long memory)")
print("    - Low ratio → Leaky integrator (short memory)")
print("    - Equal → Critically damped")

---
## 5. Steady-State Analysis

At steady state: `ds/dt = 0`, so `γ⁺·g(φ) = γ⁻·s`, giving `s* = (γ⁺/γ⁻)·g(φ)`

In [ ]:
def find_steady_state(phi_input, phi_offset, i_bias, gamma_plus, gamma_minus, tol=1e-6, max_iter=10000):
    """
    Find steady state by iterating until convergence.
    At steady state: γ⁺·g(φ, I_squid) = γ⁻·s
    But I_squid = I_bias - s, so this is implicit!
    """
    s = 0.0
    phi_total = phi_input + phi_offset
    
    for _ in range(max_iter):
        i_squid = max(i_bias - s, 0.01)
        phi_t = torch.tensor([phi_total])
        i_t = torch.tensor([i_squid])
        g_val = g_func.g(phi_t, squid_current=i_t).item()
        
        s_new = (gamma_plus / gamma_minus) * g_val
        
        if abs(s_new - s) < tol:
            return s_new, g_val
        s = s_new
    
    return s, g_val


# Compute steady-state transfer function
phi_inputs = np.linspace(-0.1, 0.4, 100)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# (0): Effect of phi_offset on transfer function
ax = axes[0]
phi_offsets = [0.02, 0.10, 0.23, 0.35]
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(phi_offsets)))

for offset, color in zip(phi_offsets, colors):
    s_steady = []
    for phi_in in phi_inputs:
        s_ss, _ = find_steady_state(phi_in, offset, i_bias=1.98, 
                                     gamma_plus=0.0005, gamma_minus=1e-6)
        s_steady.append(s_ss)
    ax.plot(phi_inputs, s_steady, color=color, lw=2, label=f'φ_offset={offset}')

ax.set_xlabel('Input Flux φ_in')
ax.set_ylabel('Steady-State Output s*')
ax.set_title('Steady-State Transfer Function\n(Effect of φ_offset)')
ax.legend()
ax.grid(True, alpha=0.3)

# (1): Effect of γ⁺/γ⁻ ratio on output magnitude
ax = axes[1]
ratios = [(0.0005, 1e-6), (0.0005, 1e-5), (0.0005, 1e-4), (0.001, 1e-4)]
labels = ['γ⁺/γ⁻=500', 'γ⁺/γ⁻=50', 'γ⁺/γ⁻=5', 'γ⁺/γ⁻=10']

for (gp, gm), label in zip(ratios, labels):
    s_steady = []
    for phi_in in phi_inputs:
        s_ss, _ = find_steady_state(phi_in, phi_offset=0.23, i_bias=1.98,
                                     gamma_plus=gp, gamma_minus=gm)
        s_steady.append(s_ss)
    ax.plot(phi_inputs, s_steady, lw=2, label=label)

ax.set_xlabel('Input Flux φ_in')
ax.set_ylabel('Steady-State Output s*')
ax.set_title('Steady-State Gain\n(Effect of γ⁺/γ⁻ Ratio)')
ax.legend()
ax.grid(True, alpha=0.3)

# (2): Effect of I_bias on saturation
ax = axes[2]
bias_currents = [1.5, 1.7, 1.9, 2.1]
colors = plt.cm.coolwarm(np.linspace(0.2, 0.8, len(bias_currents)))

for i_bias, color in zip(bias_currents, colors):
    s_steady = []
    for phi_in in phi_inputs:
        s_ss, _ = find_steady_state(phi_in, phi_offset=0.23, i_bias=i_bias,
                                     gamma_plus=0.0005, gamma_minus=1e-6)
        s_steady.append(s_ss)
    ax.plot(phi_inputs, s_steady, color=color, lw=2, label=f'I_bias={i_bias}')

ax.set_xlabel('Input Flux φ_in')
ax.set_ylabel('Steady-State Output s*')
ax.set_title('Saturation Level\n(Effect of I_bias)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSteady-State Insights:")
print("  • φ_offset shifts the threshold (where output 'turns on')")
print("  • γ⁺/γ⁻ ratio scales the output magnitude")
print("  • I_bias determines the saturation ceiling")
print("    (because I_squid = I_bias - s, when s → I_bias, I_squid → 0, g → 0)")

---
## 6. Connection Weight J - The Only Trainable Parameter

Connection weight J scales how much flux from upstream neurons reaches this neuron.

In [ ]:
# Demonstrate effect of J
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fixed upstream neuron output
s_upstream = 0.5  # Upstream neuron state

# Different connection weights
J_values = [-0.5, -0.2, 0.0, 0.2, 0.5, 1.0]

# Left: How J transforms upstream signal to flux
ax1 = axes[0]

s_up_range = np.linspace(0, 1, 100)
colors = plt.cm.RdYlGn(np.linspace(0.1, 0.9, len(J_values)))

for J, color in zip(J_values, colors):
    phi_in = J * s_up_range
    ax1.plot(s_up_range, phi_in, color=color, lw=2, label=f'J={J}')

ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax1.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
ax1.set_xlabel('Upstream State s_up')
ax1.set_ylabel('Input Flux φ_in = J · s_up')
ax1.set_title('Connection Weight J: Linear Scaling')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Full input-output with different J
ax2 = axes[1]

for J, color in zip(J_values, colors):
    s_steady = []
    for s_up in s_up_range:
        phi_in = J * s_up
        s_ss, _ = find_steady_state(phi_in, phi_offset=0.23, i_bias=1.98,
                                     gamma_plus=0.0005, gamma_minus=1e-6)
        s_steady.append(s_ss)
    ax2.plot(s_up_range, s_steady, color=color, lw=2, label=f'J={J}')

ax2.set_xlabel('Upstream State s_up')
ax2.set_ylabel('Downstream Steady-State s*')
ax2.set_title('Full Transfer Function: J Controls Sensitivity & Sign')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nConnection Weight J:")
print("  • J > 0: Excitatory connection (more upstream → more downstream)")
print("  • J < 0: Inhibitory connection (more upstream → less downstream)")
print("  • |J| large: High sensitivity to upstream")
print("  • |J| small: Low sensitivity to upstream")
print("\n  → This is the ONLY parameter learned during training!")
print("  → Physical implementation: optical attenuation/amplification")

---
## 7. 3D Visualization: Parameter Space

Visualize how multiple parameters interact.

In [ ]:
# Create 3D surface: Steady-state output as function of (φ_in, φ_offset)
phi_in_range = np.linspace(-0.1, 0.3, 50)
phi_offset_range = np.linspace(0.0, 0.4, 50)

PHI_IN, PHI_OFF = np.meshgrid(phi_in_range, phi_offset_range)
S_STEADY = np.zeros_like(PHI_IN)

for i in range(len(phi_offset_range)):
    for j in range(len(phi_in_range)):
        s_ss, _ = find_steady_state(PHI_IN[i,j], PHI_OFF[i,j], i_bias=1.98,
                                     gamma_plus=0.0005, gamma_minus=1e-6)
        S_STEADY[i,j] = s_ss

fig = plt.figure(figsize=(14, 5))

# 3D surface
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(PHI_IN, PHI_OFF, S_STEADY, cmap='viridis', alpha=0.8)
ax1.set_xlabel('Input Flux φ_in')
ax1.set_ylabel('φ_offset')
ax1.set_zlabel('Steady-State s*')
ax1.set_title('Parameter Space: s*(φ_in, φ_offset)')
fig.colorbar(surf, ax=ax1, shrink=0.5)

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contourf(PHI_IN, PHI_OFF, S_STEADY, levels=30, cmap='viridis')
ax2.contour(PHI_IN, PHI_OFF, S_STEADY, levels=[0.1, 0.5, 1.0, 1.5], colors='white', linewidths=1)
ax2.axhline(y=0.23, color='red', linestyle='--', lw=2, label='φ_offset=0.23 (threshold)')
ax2.set_xlabel('Input Flux φ_in')
ax2.set_ylabel('φ_offset')
ax2.set_title('Contour Plot: s*(φ_in, φ_offset)')
ax2.legend()
fig.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()

---
## 8. Summary: Co-Design Degrees of Freedom

In [ ]:
print("="*80)
print("SOEN CO-DESIGN SUMMARY")
print("="*80)

print("""
┌─────────────────────────────────────────────────────────────────────────────┐
│                    HARDWARE PRIMITIVE: SingleDendrite                       │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   Core Equation: ds/dt = γ⁺ · g(φ, I_squid) - γ⁻ · s                       │
│                                                                             │
│   Where: φ = Σ(J_i · s_i) + φ_offset     (total flux)                      │
│          I_squid = I_bias - s            (SQUID current)                   │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                         PARAMETER CLASSIFICATION                            │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  🔴 FIXED BY PHYSICS (Cannot change):                                       │
│     • ODE structure: ds/dt = γ⁺g - γ⁻s                                     │
│     • g(φ) periodicity and symmetry                                        │
│     • Flux quantum Φ₀                                                       │
│                                                                             │
│  🟡 FIXED AT FABRICATION (Design choices):                                  │
│     • φ_offset: Operating point (threshold position)                       │
│     • I_bias: Bias current (saturation ceiling)                            │
│     • γ⁺: Integration rate (input gain)                                    │
│     • γ⁻: Decay rate (memory time constant τ = 1/γ⁻)                       │
│     • g(φ) curve shape (SQUID geometry)                                    │
│                                                                             │
│  🟢 TRAINABLE AT RUNTIME:                                                   │
│     • J: Connection weights (optical attenuation)                          │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                         DESIGN GUIDELINES                                   │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  φ_offset ≈ 0.23:  At threshold → maximum gradient flow during training    │
│  I_bias ≈ 1.7-2.0: Provides headroom before saturation                     │
│  γ⁺/γ⁻ ratio:     Determines integrator vs leaky behavior                  │
│                    - High ratio: Long memory (good for temporal patterns)  │
│                    - Low ratio: Fast response (good for static inputs)     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
""")

print("="*80)